In [22]:
import sys
import statsmodels.api
from plotly import express as px
from sklearn import datasets
import pandas as pd

In [30]:
df= pd.read_csv('data.csv')

In [31]:
df.head()

,World Rank,Institution,Location,National Rank,Education Rank,Employability Rank,Faculty Rank,Research Rank,Score
0,1,Harvard University,USA,1,1,1,1,1,100.0
1,2,Massachusetts Institute of Technology,USA,2,4,12,2,7,96.7
2,3,Stanford University,USA,3,11,4,3,2,95.1
3,4,University of Cambridge,United Kingdom,1,3,25,4,10,94.1
4,5,University of Oxford,United Kingdom,2,7,27,9,4,93.3


In [112]:
df[df.columns[2]][0].type

AttributeError: 'str' object has no attribute 'type'

In [103]:
for i in range(0,len(df.columns)):
    print(type(df[df.columns[i]][0]))

<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>


In [ ]:
def main():
    diabetes = datasets.load_diabetes()
    X = diabetes.data
    y = diabetes.target
 
    for idx, column in enumerate(X.T):
        feature_name = diabetes.feature_names[idx]
        predictor = statsmodels.api.add_constant(column)
        linear_regression_model = statsmodels.api.OLS(y, predictor)
        linear_regression_model_fitted = linear_regression_model.fit()
        print(f"Variable: {feature_name}")
        print(linear_regression_model_fitted.summary())
 
        # Get the stats
        t_value = round(linear_regression_model_fitted.tvalues[1], 6)
        p_value = "{:.6e}".format(linear_regression_model_fitted.pvalues[1])
 
        # Plot the figure
        fig = px.scatter(x=column, y=y, trendline="ols")
        fig.update_layout(
            title=f"Variable: {feature_name}: (t-value={t_value}) (p-value={p_value})",
            xaxis_title=f"Variable: {feature_name}",
            yaxis_title="y",
        )
        fig.show()
        fig.write_html(
            file=f"../../plots/lecture_6_var_{idx}.html", include_plotlyjs="cdn"
        )
    return
 
 
if __name__ == "__main__":
    sys.exit(main())